# What's new in scipp 0.5.0

[Release Notes](https://scipp.github.io/about/release-notes.html#v0-5-january-2021)

Due to two big internal and external changes this release cycle has been very long, with many significant changes.
We generally aim for shorter cycles in the future.

Download [PG3_4844_event.nxs](http://198.74.56.37/ftp/external-data/MD5/d5ae38871d0a09a28ae01f85d969de1e) (and rename after download) to run this notebook.

- We made a `mantid-framework` MacOS `conda` package available, so installation of `scipp` in combination with Mantid is now possible without the need for a special hand-distributed package.
- Windows is still a problem, we currently recommend using WSL.

## Run logs now stored natively

- Previously we stored Mantid's `Run` and `Sample` as a scalar variable holding the Mantid object as a `PyObject`.
- `Run` is now converted into individual attributes.
- `Sample` still `PyObject`

In [1]:
import scipp as sc
if not sc.__version__.startswith('0.5'):
    print(f'This notebook was made for scipp-0.5 and will likely not work with your version ({sc.__version__}).')

<IPython.core.display.Javascript object>

In [2]:
filename = 'PG3_4844_event'
data = sc.neutron.load(filename=f'{filename}.nxs')
data

Workspace run log 'proton_charge' has unrecognised units: 'picoCoulombs'
Workspace run log 'gd_prtn_chrg' has unrecognised units: 'uA.hour'


<scipp.DataArray>
Dimensions: {{spectrum, 24794}, {tof, 1}}
Coordinates:
  position                  vector_3_float64  [m]              (spectrum)  [(1.174510, -1.011061, -2.037967), (1.181476, -0.959466, -2.053341), ..., (1.814290, 0.095658, 3.843383), (1.813751, 0.149937, 3.842696)]
  sample-position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, 0.000000)]
  source-position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, -60.000000)]
  spectrum                  int32      [dimensionless]  (spectrum)  [1, 2, ..., 24793, 24794]
  tof                       float64    [μs]            (tof [bin-edge])  [19.000000, 16694.300781]
Data:
                            DataArrayView  [dimensionless]  (spectrum, tof)  [<scipp.DataArrayView>
Dimensions: {{event, 0}}
Coordinates:
  pulse-time                int64      [ns]             (event)  []
  tof                       float64    [μs]            (event)  []
Data:
                            float32    [counts]         (event)  []  []

, <scipp.DataArrayView>
Dimensions: {{event, 0}}
Coordinates:
  pulse-time                int64      [ns]             (event)  []
  tof                       float64    [μs]            (event)  []
Data:
                            float32    [counts]         (event)  []  []

, ..., <scipp.DataArrayView>
Dimensions: {{event, 0}}
Coordinates:
  pulse-time                int64      [ns]             (event)  []
  tof                       float64    [μs]            (event)  []
Data:
                            float32    [counts]         (event)  []  []

, <scipp.DataArrayView>
Dimensions: {{event, 0}}
Coordinates:
  pulse-time                int64      [ns]             (event)  []
  tof                       float64    [μs]            (event)  []
Data:
                            float32    [counts]         (event)  []  []

]
Attributes:
  ChopperStatus1            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}}
Coordinates:
  time                      int64      [ns]             (time)  [1313164216800000000, 1313169725000000000]
Data:
                            float64    [dimensionless]  (time)  [4.000000, 4.000000]

]
  ChopperStatus2            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}}
Coordinates:
  time                      int64      [ns]             (time)  [1313164216816000000, 1313169725000000000]
Data:
                            float64    [dimensionless]  (time)  [4.000000, 4.000000]

]
  ChopperStatus3            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}}
Coordinates:
  time                      int64      [ns]             (time)  [1313164216816000000, 1313169725000000000]
Data:
                            float64    [dimensionless]  (time)  [4.000000, 4.000000]

]
  CurrentSP                 DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}}
Coordinates:
  time                      int64      [ns]             (time)  [1313164216800000000, 1313169725000000000]
Data:
                            float64    [dimensionless]  (time)  [300.000000, 300.000000]

]
  EnergyRequest             DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}}
Coordinates:
  time                      int64      [ns]             (time)  [1313164216784000000, 1313169725000000000]
Data:
                            float64    [meV]            (time)  [287.955414, 287.955414]

]
  Filename                  string     [dimensionless]  ()  ["/home/simon/jupyter/PG3_4844_event.nxs"]
  LKSRampRate               DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}}
Coordinates:
  time                      int64      [ns]             (time)  [1313164216800000000, 1313169725000000000]
Data:
                            float64    [dimensionless]  (time)  [0.000000, 0.000000]

]
  LambdaRequest             DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}

Many time-dependent logs are just data arrays, so we can access, modify, plot, ... them.
Remember that the `value` property has to be used to access the only value of a scalar variable:

In [3]:
sc.plot.plot(data.attrs['proton_charge'].value, marker='.')

## Improved instrument view

- Align camera to axis buttons
- Cut surface tools
- Bug fixes, such as pixel sizes on high-res displays

In [4]:
sc.neutron.instrument_view(data)

## Improved plotting

- `logx`, `logy`, `log`, and transpose buttons in toolbar
- now available as method in addition to free function, `data.plot()` instead of `sc.plot.plot(data)`
- profile plot/picking for 3-D (and higher) data
- Cleaned up widgets for 3-D (and higher) data
- Performance improvements
- Dynamic event data resampling (see below)

In [5]:
data.plot()

In [6]:
# This cell is just for setup, ignore, or see below for `sc.bin`
import numpy as np
time = data.bins.data.coords['pulse-time']
start = sc.min(time).value
stop = sc.max(time).value
time_edges = sc.array(dims=['pulse-time'], unit=time.unit, values=np.linspace(start, stop, num=100))
tof_edges = sc.array(dims=['tof'], unit=sc.units.us, values=np.linspace(0,17000, num=100))
binned = sc.bin(data, edges=[time_edges])
hist = sc.histogram(binned, tof_edges)

Slider and "Profile" button show up when data is 3-D (or higher):

In [7]:
hist.plot(axes={'x':'tof', 'y':'spectrum'})

## Creation functions

For convenience and similarity to `numpy` we added [functions that create variables](https://scipp.github.io/python-reference/api.html#creation-functions).
Our intention is to fully replace the need to use `sc.Variable` directly, but at this point this has not been rolled out to our documentation pages.

In [8]:
sc.array(dims=['x'], values=np.array([1,2,3]))

<scipp.Variable>          int64      [dimensionless]  (x)  [1, 2, 3]

In [9]:
sc.zeros(dims=['x'], shape=[3])

<scipp.Variable>          float64    [dimensionless]  (x)  [0.000000, 0.000000, 0.000000]

In [10]:
sc.scalar(17)

<scipp.Variable>          int64      [dimensionless]  ()  [17]

All of these also take keyword arguments.
Note that we can still support creating scalars by multiplying with a unit:

In [11]:
1.2 * sc.units.m

<scipp.Variable>          float64    [m]              ()  [1.200000]

## Label-based indexing

- Works just like position indexing (with integers).
- Use a scalar variable as index (instead of integer) to use label-based indexing
- Works with single values as well as slices (`:` notation)

See [Label-based indexing](https://scipp.github.io/user-guide/slicing.html#Label-based-indexing) for more details.

In [12]:
sc.to_html(hist['spectrum', sc.scalar(1000)])
sc.to_html(hist['tof', 1000*sc.units.us]) # selects bin containing this value

## Dataset items and masks

- Previously masks for global for a dataset, masking items individually was not supported.
- Applications require item-specific masks.
- Removed support for dataset masks, only item masks are supported now, since having both causes conceptual problems.
- See [ADR0001](https://scipp.github.io/developer/adr/0001-remove-dataset-masking-support.html) for details.
- Common masks must be set individually.

In [13]:
hist = sc.histogram(data, tof_edges)
ds = sc.Dataset({'sample1':hist, 'sample2':hist+hist})
tof_mask = ds.coords['tof']['tof',:-1] < 3333*sc.units.us
ds['sample1'].masks['tof'] = tof_mask
ds['sample2'].masks['tof'] = tof_mask
ds['sample2'].masks['tof2'] = ~tof_mask
ds

<scipp.Dataset>
Dimensions: {{spectrum, 24794}, {tof, 99}}
Coordinates:
  position                  vector_3_float64  [m]              (spectrum)  [(1.174510, -1.011061, -2.037967), (1.181476, -0.959466, -2.053341), ..., (1.814290, 0.095658, 3.843383), (1.813751, 0.149937, 3.842696)]
  sample-position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, 0.000000)]
  source-position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, -60.000000)]
  spectrum                  int32      [dimensionless]  (spectrum)  [1, 2, ..., 24793, 24794]
  tof                       float64    [μs]            (tof [bin-edge])  [0.000000, 171.717172, ..., 16828.282828, 17000.000000]
Data:
  sample1                   float32    [counts]         (spectrum, tof)  [0.000000, 0.000000, ..., 0.000000, 0.000000]  [0.000000, 0.000000, ..., 0.000000, 0.000000]
    Masks:
        tof                       bool       [dimensionless]  (tof)  [True, True, ..., False, False]
    Attributes:
        ChopperStatus1            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}}
Coordinates:
  time                      int64      [ns]             (time)  [1313164216800000000, 1313169725000000000]
Data:
                            float64    [dimensionless]  (time)  [4.000000, 4.000000]

]
        ChopperStatus2            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}}
Coordinates:
  time                      int64      [ns]             (time)  [1313164216816000000, 1313169725000000000]
Data:
                            float64    [dimensionless]  (time)  [4.000000, 4.000000]

]
        ChopperStatus3            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}}
Coordinates:
  time                      int64      [ns]             (time)  [1313164216816000000, 1313169725000000000]
Data:
                            float64    [dimensionless]  (time)  [4.000000, 4.000000]

]
        CurrentSP                 DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}}
Coordinates:
  time                      int64      [ns]             (time)  [1313164216800000000, 1313169725000000000]
Data:
                            float64    [dimensionless]  (time)  [300.000000, 300.000000]

]
        EnergyRequest             DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}}
Coordinates:
  time                      int64      [ns]             (time)  [1313164216784000000, 1313169725000000000]
Data:
                            float64    [meV]            (time)  [287.955414, 287.955414]

]
        Filename                  string     [dimensionless]  ()  ["/home/simon/jupyter/PG3_4844_event.nxs"]
        LKSRampRate               DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}}
Coordinates:
  time                      int64      [ns]             (time)  [1313164216800000000, 1313169725000000000]
Data:
                            float64    [dimensionless]  (time)  [0.000000, 0.000000]

]
        LambdaRequest             DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}}
Coordinates:
  time                      int64      [ns]             (time)  [1313164216784000000, 1313169725000000000]
Data:
                            float64    [Å]            (time)  [0.533000, 0.533000]

]
        Phase1                    DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 1794}}
Coordinates:
  time                      int64      [ns]             (time)  [1313164216800000000, 1313164218267000007, ..., 1313169722999218750, 1313169725000000000]
Data:
                            float64    [μs]            (time)  [8166.720215, 8165.158203, ..., 8163.852539, 8163.852539]

]
        Phase2                    DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 1793}}
Coordinates:
  time                      int64      [ns]             (time)  [131316421680000

## HDF5 I/O

Scipp can now [write and read its data structures](https://scipp.github.io/user-guide/reading-and-writing-files.html) using `h5py`.
- At this point we do not make any promises about compatibilty of the file layout with future version, so we do not encourage usage for storing data long-term.
- Loading event data via Mantid converter is quite slow, so loading raw data once and storing natively can provide a UX boost when working on debugging a reduction workflow in a notebook.
- Note the warning message about `PyObject`, since we cannot write Mantid's `Sample` object, apart from this caveat everything else is saved.

In [14]:
data.to_hdf5(filename=f'{filename}-scipp.hdf5')

Writing with dtype=PyObject not implemented, skipping.


In [15]:
data2 = sc.io.open_hdf5(filename=f'{filename}-scipp.hdf5')
data2

<scipp.DataArray>
Dimensions: {{spectrum, 24794}, {tof, 1}}
Coordinates:
  position                  vector_3_float64  [m]              (spectrum)  [(1.174510, -1.011061, -2.037967), (1.181476, -0.959466, -2.053341), ..., (1.814290, 0.095658, 3.843383), (1.813751, 0.149937, 3.842696)]
  sample-position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, 0.000000)]
  source-position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, -60.000000)]
  spectrum                  int32      [dimensionless]  (spectrum)  [1, 2, ..., 24793, 24794]
  tof                       float64    [μs]            (tof [bin-edge])  [19.000000, 16694.300781]
Data:
                            DataArrayView  [dimensionless]  (spectrum, tof)  [<scipp.DataArrayView>
Dimensions: {{event, 0}}
Coordinates:
  pulse-time                int64      [ns]             (event)  []
  tof                       float64    [μs]            (event)  []
Data:
                            float32    [counts]         (event)  []  []

, <scipp.DataArrayView>
Dimensions: {{event, 0}}
Coordinates:
  pulse-time                int64      [ns]             (event)  []
  tof                       float64    [μs]            (event)  []
Data:
                            float32    [counts]         (event)  []  []

, ..., <scipp.DataArrayView>
Dimensions: {{event, 0}}
Coordinates:
  pulse-time                int64      [ns]             (event)  []
  tof                       float64    [μs]            (event)  []
Data:
                            float32    [counts]         (event)  []  []

, <scipp.DataArrayView>
Dimensions: {{event, 0}}
Coordinates:
  pulse-time                int64      [ns]             (event)  []
  tof                       float64    [μs]            (event)  []
Data:
                            float32    [counts]         (event)  []  []

]
Attributes:
  ChopperStatus1            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}}
Coordinates:
  time                      int64      [ns]             (time)  [1313164216800000000, 1313169725000000000]
Data:
                            float64    [dimensionless]  (time)  [4.000000, 4.000000]

]
  ChopperStatus2            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}}
Coordinates:
  time                      int64      [ns]             (time)  [1313164216816000000, 1313169725000000000]
Data:
                            float64    [dimensionless]  (time)  [4.000000, 4.000000]

]
  ChopperStatus3            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}}
Coordinates:
  time                      int64      [ns]             (time)  [1313164216816000000, 1313169725000000000]
Data:
                            float64    [dimensionless]  (time)  [4.000000, 4.000000]

]
  CurrentSP                 DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}}
Coordinates:
  time                      int64      [ns]             (time)  [1313164216800000000, 1313169725000000000]
Data:
                            float64    [dimensionless]  (time)  [300.000000, 300.000000]

]
  EnergyRequest             DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}}
Coordinates:
  time                      int64      [ns]             (time)  [1313164216784000000, 1313169725000000000]
Data:
                            float64    [meV]            (time)  [287.955414, 287.955414]

]
  Filename                  string     [dimensionless]  ()  ["/home/simon/jupyter/PG3_4844_event.nxs"]
  LKSRampRate               DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}}
Coordinates:
  time                      int64      [ns]             (time)  [1313164216800000000, 1313169725000000000]
Data:
                            float64    [dimensionless]  (time)  [0.000000, 0.000000]

]
  LambdaRequest             DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}

## Event data

- Event data is the main use case of the new [Binned Data](https://scipp.github.io/user-guide/binned-data.html) support.
- Old special "event list" `dtype` and "realigned data support" have been replaced and removed.
- Much more flexible and generic, with a more natural interface.
  - Will be able to support multi-dimensional event data with this new data layout (for single-crystal and inelastic data).
- Plotting with on-the-fly resampling.

See documentation for more details and how to work with binned event data.

In [16]:
data

<scipp.DataArray>
Dimensions: {{spectrum, 24794}, {tof, 1}}
Coordinates:
  position                  vector_3_float64  [m]              (spectrum)  [(1.174510, -1.011061, -2.037967), (1.181476, -0.959466, -2.053341), ..., (1.814290, 0.095658, 3.843383), (1.813751, 0.149937, 3.842696)]
  sample-position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, 0.000000)]
  source-position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, -60.000000)]
  spectrum                  int32      [dimensionless]  (spectrum)  [1, 2, ..., 24793, 24794]
  tof                       float64    [μs]            (tof [bin-edge])  [19.000000, 16694.300781]
Data:
                            DataArrayView  [dimensionless]  (spectrum, tof)  [<scipp.DataArrayView>
Dimensions: {{event, 0}}
Coordinates:
  pulse-time                int64      [ns]             (event)  []
  tof                       float64    [μs]            (event)  []
Data:
                            float32    [counts]         (event)  []  []

, <scipp.DataArrayView>
Dimensions: {{event, 0}}
Coordinates:
  pulse-time                int64      [ns]             (event)  []
  tof                       float64    [μs]            (event)  []
Data:
                            float32    [counts]         (event)  []  []

, ..., <scipp.DataArrayView>
Dimensions: {{event, 0}}
Coordinates:
  pulse-time                int64      [ns]             (event)  []
  tof                       float64    [μs]            (event)  []
Data:
                            float32    [counts]         (event)  []  []

, <scipp.DataArrayView>
Dimensions: {{event, 0}}
Coordinates:
  pulse-time                int64      [ns]             (event)  []
  tof                       float64    [μs]            (event)  []
Data:
                            float32    [counts]         (event)  []  []

]
Attributes:
  ChopperStatus1            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}}
Coordinates:
  time                      int64      [ns]             (time)  [1313164216800000000, 1313169725000000000]
Data:
                            float64    [dimensionless]  (time)  [4.000000, 4.000000]

]
  ChopperStatus2            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}}
Coordinates:
  time                      int64      [ns]             (time)  [1313164216816000000, 1313169725000000000]
Data:
                            float64    [dimensionless]  (time)  [4.000000, 4.000000]

]
  ChopperStatus3            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}}
Coordinates:
  time                      int64      [ns]             (time)  [1313164216816000000, 1313169725000000000]
Data:
                            float64    [dimensionless]  (time)  [4.000000, 4.000000]

]
  CurrentSP                 DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}}
Coordinates:
  time                      int64      [ns]             (time)  [1313164216800000000, 1313169725000000000]
Data:
                            float64    [dimensionless]  (time)  [300.000000, 300.000000]

]
  EnergyRequest             DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}}
Coordinates:
  time                      int64      [ns]             (time)  [1313164216784000000, 1313169725000000000]
Data:
                            float64    [meV]            (time)  [287.955414, 287.955414]

]
  Filename                  string     [dimensionless]  ()  ["/home/simon/jupyter/PG3_4844_event.nxs"]
  LKSRampRate               DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}}
Coordinates:
  time                      int64      [ns]             (time)  [1313164216800000000, 1313169725000000000]
Data:
                            float64    [dimensionless]  (time)  [0.000000, 0.000000]

]
  LambdaRequest             DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 2}

In [17]:
sc.table(data.values[7]['event',:10])

In [18]:
data.plot()